# Unlocking the Potential of Large Language Models with LangChain


[LangChain](https://python.langchain.com/en/latest/getting_started/getting_started.html) is a widely used framework that enables users to easily develop applications and pipelines using Large Language Models (LLMs). With LangChain, you can create chatbots, Generative Question-Answering (GQA) systems, summarization tools, and much more.

At the core of LangChain's design is the idea of "chaining" together different components to create more sophisticated LLM use-cases. These chains can comprise multiple components from various modules, including:

  * ***Prompt Templates***: serve as pre-defined structures for different types of prompts such as chatbot-style interactions and more.

  * ***Large Language Models***: LLMs such as GPT-3, BLOOM and Jurassic are used in LangChain to generate responses or perform language-related tasks.

  * ***Agents***: utilize LLMs to determine the appropriate actions to take, using tools such as web search or calculators to execute operations within a logical loop.

  * ***Memory***: LangChain provides both short-term and long-term memory to help LLMs retain information and improve their responses over time.

With these modules, LangChain offers a comprehensive solution for developing advanced LLM-based applications that can enhance natural language processing in various fields.

The following code downloads the necessary python libraries needed in this LAB, and then it load them.

In [1]:
!pip install -qU langchain pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.6 MB/s eta 0:00:00


In [32]:
import requests
from langchain.llms import AI21
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

Afterwards, the API to the Large Language Models provided by [AI21](https://studio.ai21.com) is initizalized.

> If not working, go to the linked site and make an account. You have to insert your own API key provided in the site.

In [33]:
AI21_API_KEY = "4tzuSshBbSZhol8DJjRPTvgq5SF1Xoeu"
llm = AI21(ai21_api_key=AI21_API_KEY)

## Structure of a Prompt

A prompt can consist of multiple components:

* **Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.
* **Additional information or context** that is either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).
* **User input or query** is typically a query directly input by the user of the system.
* **Output indicator** is the *beginning* of the generated text.

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

In [34]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991.
It's known for its clear syntax, readability, and simplicity, which makes it a popular language for beginners to learn programming.
However, it's also widely used in scientific computing, data analysis, artificial intelligence, web development, and many other areas.

Key features of Python include:
  1) Interpreted: Python code doesn't need to be compiled before it's run, which makes the write-test-debug cycle very fast.
  2) Dynamically Typed: In Python, you don't have to declare the data type of a variable. The type is determined at runtime.
  3) Object-Oriented: Python supports object-oriented programming which allows data structures to be re-used.
  4) Extensive Libraries: Python has a large standard library that includes areas like internet protocols, string operations, web services tools and operating system interfaces.
     Many high-quality libraries (like NumPy, Pandas, and Matplotlib) are available for Python, covering a wide range of applications, from web development to machine learning.
  5) Indentation syntax: Python uses whitespace indentation, rather than curly braces or keywords, to delimit blocks—an unusual trait among popular programming languages.
  6) Garbage Collection: Python's memory management is handled by the language itself, meaning that developers generally don't need to worry about allocating and deallocating memory manually.

These features make Python a versatile language that's used across a range of different industries and in a variety of different roles,
from web and game development to scientific research, data analysis, and AI development.

Question: Which are the main features of Python?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```



In [35]:
prompt = """Dove si trova Reggio Emilia?"""
print(llm.invoke(prompt))


Reggio Emilia is located in northern Italy, about 40 miles south of Milan.


## Prompt Templates

In [36]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991.
It's known for its clear syntax, readability, and simplicity, which makes it a popular language for beginners to learn programming.
However, it's also widely used in scientific computing, data analysis, artificial intelligence, web development, and many other areas.

Key features of Python include:
  1) Interpreted: Python code doesn't need to be compiled before it's run, which makes the write-test-debug cycle very fast.
  2) Dynamically Typed: In Python, you don't have to declare the data type of a variable. The type is determined at runtime.
  3) Object-Oriented: Python supports object-oriented programming which allows data structures to be re-used.
  4) Extensive Libraries: Python has a large standard library that includes areas like internet protocols, string operations, web services tools and operating system interfaces.
     Many high-quality libraries (like NumPy, Pandas, and Matplotlib) are available for Python, covering a wide range of applications, from web development to machine learning.
  5) Indentation syntax: Python uses whitespace indentation, rather than curly braces or keywords, to delimit blocks—an unusual trait among popular programming languages.
  6) Garbage Collection: Python's memory management is handled by the language itself, meaning that developers generally don't need to worry about allocating and deallocating memory manually.

These features make Python a versatile language that's used across a range of different industries and in a variety of different roles,
from web and game development to scientific research, data analysis, and AI development.

Question: {question}

Answer: """

# Make the prompt template
prompt = ChatPromptTemplate.from_template(template=template)

# Create a chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# What would you like to ask?
question = "What does it mean that has Indentation syntax?"
response = llm_chain.invoke({'question': question})
print('Q:', response['question'])
print(response['text'])

Q: What does it mean that has Indentation syntax?
Indentation syntax means that Python uses whitespace indentation, rather than curly braces or keywords, to delimit blocks.


If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template ("question") that is mapped to the question we'd like to ask.

In [37]:
# Make a list of questions
qs = [
    {'question': "Cos'e' un Gargbage Collection?"},
    {'question': "What is a akdgjsf syntax?"},
    {'question': "Can you use python to conver english to French?"},
    {'question': "Where is Reggio Emilia?"}
]

# Use apply for only receiving answers
res = llm_chain.apply(qs)
for i, r in enumerate(res):
  print(f"{i+1}) {qs[i]['question'].strip()}")
  print(r['text'].strip(), '\n')

1) Cos'e' un Gargbage Collection?
In Python, garbage collection is handled by the language itself, meaning that developers generally don't need to worry about allocating and deallocating memory manually. 

2) What is a akdgjsf syntax?
Indentation syntax refers to the use of whitespace, rather than curly braces or keywords, to delimit blocks. 

3) Can you use python to conver english to French?
Python cannot be used directly to translate English text into French. However, there are packages and libraries available that allow developers to use Python for Natural Language Processing (NLP) tasks such as machine translation. 

4) Where is Reggio Emilia?
I don't know 



#LLMs agents can use tools to improve their answer



## Python REPL

Sometimes, for complex calculations, rather than have an LLM generate the answer directly, it can be better to have the LLM generate code to calculate the answer, and then run that code to get the answer. In order to easily do that, we provide a simple Python REPL to execute commands in.

This interface will only return things that are printed - therefore, if you want to use it to calculate an answer, make sure to have it print out the answer.

In [38]:
from langchain.utilities import PythonREPL

# Create the python tool
python_repl = PythonREPL()

# Run the commands
python_repl.run('''
a=35
b=9
c=a*b
print(c)
''')

'315\n'

In [41]:
from langchain.agents import initialize_agent

# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="This tool is a Python shell. Use it to execute python commands. "\
                "The input should be a valid python command and inside your code you should print what you want to observe. "\
                "For example, if you want to observe the result of 89 * 76, you can do as follows: "\
                "'''a=89\nb=76\nprint(a*b)'''. ",
    func=python_repl.run
)

# Pass the tool to the agent
agent = initialize_agent([repl_tool], llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=2)

In [42]:
#
agent.invoke("How much is 35 times 9?")



> Entering new AgentExecutor chain...
 I know that 35 * 9 = 315
Action: python_repl
Action Input: 35 * 9
Observation: 
Thought: 
315

Final Answer: 315

> Finished chain.


{'input': 'How much is 35 times 9?', 'output': '315'}

### Task for today's LAB

Try to give a good prompt such that the LLM agent is able to print result of 132 * 45

In [43]:
# We create the tool
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. "\
                "The input should be a valid python command inside the print function. ",
    func=python_repl.run
)
agent = initialize_agent([repl_tool], llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=2)

# Which question?
agent("Put your code here")



> Entering new AgentExecutor chain...
 I'll use the python_repl to execute a python code
Action: [python_repl]
Action Input: print("Hello, World!")
Observation: [python_repl] is not a valid tool, try one of [python_repl].
Thought: I'll use the python_repl to execute a python code
Action: [python_repl]
Action Input: print("Hello, World!")
Observation: [python_repl] is not a valid tool, try one of [python_repl].
Thought:

> Finished chain.


{'input': 'Put your code here',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Internet Requests

Use Google to find answers.

In [48]:
from langchain.chains import LLMRequestsChain

# Create the template
template="""Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""
PROMPT = ChatPromptTemplate.from_template(template)

# Create the request chain
chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))

# Ask the query
question = "What are the Three (3) biggest countries, and their respective sizes?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
chain(inputs)

{'query': 'What are the Three (3) biggest countries, and their respective sizes?',
 'url': 'https://www.google.com/search?q=What+are+the+Three+(3)+biggest+countries,+and+their+respective+sizes?',
 'output': 'The three biggest countries are Russia, Canada, and the United States.'}

## Math

Similar to Python REPL but with a built-in chain.

In [49]:
from langchain.chains import LLMMathChain

# Create the math chain
llm_math = LLMMathChain.from_llm(llm, verbose=True)

# Ask
llm_math.run("What is 13 raised to the .3432 power?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13 ** .3432
```

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

## Document understanding and summarization

We are going to summarize documents on it's own in a "map" step and then "reduce" the summaries into a final summary

> [For more advanced options see here](https://python.langchain.com/v0.1/docs/use_cases/summarization/)

We start by downloading a pdf from the net and summarize it.

In [50]:
# Get the pdf
url = 'https://arxiv.org/pdf/1706.03762.pdf' # replace with your url
response = requests.get(url)

# Get the file name from the url, if it's available
file_name = url.split("/")[-1] if '/' in url else 'output.pdf'

# Save the file to a current file (for checking results)
with open(file_name, 'wb') as file:
    file.write(response.content)

In [52]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader(file_name)
pages = loader.load()

# Cut out the open and closing parts
# pages = pages[26:277]

# Combine the pages, and replace the tabs with spaces
text = ""
for page in pages:
    text += page.page_content
text = text.replace('\t', ' ')
text[:100]

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\n'

In [53]:
#
num_tokens = llm.get_num_tokens(text)
print (f"This book has {num_tokens} tokens in it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (10540 > 1024). Running this sequence through the model will result in indexing errors


This book has 10540 tokens in it


In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"],
                                               chunk_size=4096,
                                               chunk_overlap=500)
docs = text_splitter.create_documents([text])

#
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)
print (f"Now we have {len(docs)} documents and the first one has {num_tokens_first_doc} tokens")

Now we have 11 documents and the first one has 938 tokens


In [59]:
from langchain.chains.summarize import load_summarize_chain

#
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt,
                                     input_variables=["text"])


#
combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt,
                                         input_variables=["text"])

# Summarize chain
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template)
output = summary_chain.run(docs)

In [60]:
print (output)


* "Attention Is All You Need" is a research paper by Ashish Vaswani et al. proposing a new simple network architecture, the Transformer, based solely on attention mechanisms.
* This architecture outperforms other models in quality, while being more parallelizable and requiring less time to train.
* The model is composed of a stack of N= 6 identical layers, each with a multi-head self-attention mechanism followed by a feed-forward network.
* Residual connections and layer normalization are used around each sub-layer.
* The model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task and 41.8 BLEU on the WMT 2014 English-to-French translation task.
* The Transformer model achieves better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
* The model uses multi-head attention to jointly attend to information from different representation subspaces at different positions.
* 

# Building Tools

[LLM agents](https://pinecone.io/learn/langchain-agents) are one of the most powerful and fascinating technologies to come out of the huge explosion of LLMs.

By employing agents, we can equip Large Language Models (LLMs) with a wide array of tools. This equipping strategy unfolds virtually limitless possibilities, as it empowers us to conduct web searches, perform calculations, execute code, and much more.

LangChain provides a vast assortment of prebuilt tools. However, in numerous real-world scenarios, it becomes necessary to craft bespoke tools tailored to meet the specific needs of our use cases.
